# This generates a file containing the MJ background estimation from the ABCD method:
## A/B = C/D 
Where,
A = Signal Region (SR)
B = Signal Region Same Sign (SR-SS)
C = Control Region (CR)
D = Control Region Same Sign (CR-SS)

$$\frac{\text{SR}}{\text{SR}_{\text{SS}}} = \frac{\text{CR}}{\text{CR}_{\text{SS}}}$$


$$ \text{SR} = \text{SR}_{\text{SS}} \text{RQCD} $$

### Signal region path$
### Multi jet control region path
### Sample used for Z-QCDjets

In [12]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 
from histogramHelpers import *


# Define the path to the channel and the directories for the regions.
channelPath = "/Users/user/Documents/HEP/VBF-Analysis/VBFAnalysisPlots/TauTau/TauhadTaulep/Higgs/"
SR = "NewBDTCutSR/"
SRSS = "NewBDTCutSRSS/"
CR = "NonIsoOS/"
CRSS = "NonIsoSS/"

# Select EWjj and QCDjj samples
EWjj = "Signal_Sherpa"
QCDjj = "Ztautau_SherpaRW"

RQCDwithCRs = False

# Remove previous MJ.root file
samples=[i for i in os.listdir(channelPath+SR) if ('.root' in i and 'Wjets' not in i)]
print(samples)
if "MJ.root" in samples:
    os.system("rm "+channelPath+SR+"MJ.root")

['MJ.root', 'Ztautau_MGNLORW.root', 'Ztautau_SherpaRW.root', 'Ztautau_MGRW.root', 'Zjets.root', 'Signal_Sherpa.root', 'W_EWK_PoPy.root', 'Ztautau_SherpaNLORW.root', 'Signal_PoPy.root', 'Data.root', 'MC.root', 'VV.root', 'Higgs.root', 'VV_EWK.root', 'W_EWK_Sherpa.root', 'ttbar.root', 'singletop.root']


In [13]:
import sys
sys.path.append('/Users/user/Documents/HEP/VBF-Analysis/Scripts/')
from histogramHelpers import tautauHighMassHistograms,tautauHighMassMJHistograms,emuHighMassHistograms,tautauZpeakHistograms,tautauHiggsBDTHistograms,tautauHiggsHistograms
histos = tautauHiggsBDTHistograms

In [14]:
mcSamples = [EWjj,QCDjj]
backgroundSamples = ['VV',"ttbar",'singletop']#+['Wjets',]
if ("Tau" in channelPath) or ("MuEle" in channelPath):
    backgroundSamples += ['Higgs','Zjets','W_EWK_Sherpa','VV_EWK']
mcSamples += backgroundSamples

# Open target file
multiJetFile =r.TFile.Open(channelPath+SR+"MJ.root", "RECREATE")

print('Using the following samples to perform the data subtraction: \n',mcSamples)

# First evaluate if there is evidence for MJ BG.
totalMJisZero = False
nGapJetsHistogram = dataSubtract("n_jets_interval",channelPath+SRSS,"Data",mcSamples,histos,rebin=False)
totalMJ = nGapJetsHistogram.GetBinContent(1)
totalMJUncertainty = nGapJetsHistogram.GetBinError(1)
if (totalMJ < 0.0) or (totalMJ/totalMJUncertainty < 1.0 ):
    totalMJisZero = True
    print("NO MJ EVIDENCE!")
print("MJ = ",totalMJ," +- ",totalMJUncertainty)

if totalMJ < 0.0:
    totalMJ = 0.0
    

Using the following samples to perform the data subtraction: 
 ['Signal_Sherpa', 'Ztautau_SherpaRW', 'VV', 'ttbar', 'singletop', 'Higgs', 'Zjets', 'W_EWK_Sherpa', 'VV_EWK']
MJ =  2.153170108795166  +-  2.029197464297333


In [15]:
for hist in histos:
    # Determine if histogram needs to be rebined
    rebinHistogram = hist.needsRebin()
    
    print(hist.m_name,rebinHistogram)
    
    # Calculate RQCD in MJCRs
    if RQCDwithCRs:
        # Calculate RQCD
        dataSubtractedHistoCR = dataSubtract(hist.m_name,channelPath+CR,"Data",mcSamples,hist,rebin=rebinHistogram)
        dataSubtractedHistoCRSS = dataSubtract(hist.m_name,channelPath+CRSS,"Data",mcSamples,hist,rebin=rebinHistogram)
        makeNegativeBinsZero(dataSubtractedHistoCR)
        makeNegativeBinsZero(dataSubtractedHistoCRSS)
        RQCD = dataSubtractedHistoCR.Integral(1,-1)/dataSubtractedHistoCRSS.Integral(1,-1)
    # If not, use input value
    else:
        RQCD = 1.2
        UncerRQCD = 1.2*0.25
    
    # Calculate the MJ Background shape
    dataSubtractedHistoSRSS = dataSubtract(hist.m_name,channelPath+SRSS,"Data",mcSamples,hist,rebin=rebinHistogram)
    dataSubtractedHistoSRSS.Scale(RQCD)
    makeNegativeBinsZero(dataSubtractedHistoSRSS)
    MJ = dataSubtractedHistoSRSS.Clone()
    
    # If no MJ Evidence fix the bins that are in the final selection
    if totalMJisZero:
        SRhisto = mcAdd(hist.m_name,channelPath+SR,backgroundSamples,hist,rebin=rebinHistogram)
        cutLeft = hist.m_leftCut
        cutRight = hist.m_rightCut
        makeSRBinsConsistentWithNOMJ(MJ,cutLeft,cutRight,totalMJ,totalMJUncertainty,SRhisto)
        
    # Take into account the uncertainty in RQCD
    scaleUncertainty(MJ,UncerRQCD/RQCD)
    
    # Save histogram in MJ.root file
    multiJetFile.WriteObject(MJ,hist.m_name)
    
    
multiJetFile.Close()

n_bjets False
lepiso False
n_jets_interval False
flavourJet1_basic_all False
flavourJet2_basic_all False
elecPdgID_basic_all False
muonPdgID_basic_all False
tauPdgID_basic_all False
nLightJets_basic_all False
tau_pt True
lep_pt True
delta_phi True
delta_y True
omega True
rnn_score_1p True
rnn_score_3p True
ljet0_pt True
ljet1_pt True
pt_bal True
Z_centrality True
mass_jj True
reco_mass_i True
reco_mass_o True
reco_mass_ True
Z_pt_reco_i_basic_all True
Z_pt_reco_o_basic_all True
ratio_zpt_sumjetpt_basic_all True
vec_sum_pt_jets_basic_all True
moreCentralJet_basic_all True
lessCentralJet_basic_all True
delta_phijj_basic_all True
massTauClosestJet_basic_all True
massLepClosestJet_basic_all True
massTauFurthestJet_basic_all True
nuTauPt True
nuPtAssummetry_basic_all True
bdtScore True
lepNuPt True
pTsymmetry True
lepTransMass True
tauTransMass_basic_all True
signedCentrality_basic_all True
visibleMass_basic_all True
recoVisibleMassRatio True


Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 5 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 6 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 5 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 6 of r

In [22]:
from ctypes import *
osE = c_double(0.0)
ssE = c_double(0.0)
osHisto = dataSubtract("mass_jj","/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/Z-Peak/MJCROS/","Data",mcSamples,histos,rebin=True)
ssHisto = dataSubtract("mass_jj","/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/Z-Peak/MJCRSS/","Data",mcSamples,histos,rebin=True)
osIntegral = osHisto.IntegralAndError(1,-1,osE)
ssIntegral = ssHisto.IntegralAndError(1,-1,ssE)

print("OS: ",osIntegral)
print("SS: ",ssIntegral)

osE = osE.value
ssE = ssE.value

print("OS E: ",osE)
print("SS E: ",ssE)
print(osIntegral/ssIntegral," +- ",osIntegral/ssIntegral*np.sqrt((osE/osIntegral)**2+(ssE/ssIntegral)))

OS:  94.53706323355436
SS:  88.52085138997063
OS E:  19.725730909960834
SS E:  10.652698160888127
1.0679637819690624  +-  0.43233208949062885
